# Abstraction Invariant Hint Production Script
Author: Thomas Willingham  
  
This script is a utility which allows you to easily isolate the values that should be added to a high fidelity CNF file to set the AI variables to be the same as they were in a lower-fidelity run of CBMC over the same protocol. 

## Names of Files
`small_cnf` is an input file, containing the output of a CBMC run with the --dimacs option, and the content of the file before the CNF problem statement removed. This CNF should be for the version of the problem with a smaller number of bits.  
`small_sat_output` is an input file, containing the output of a suitable SAT solver on the `small_cnf` CNF.  
`big_cnf` is the same as `small_cnf`, but for the version of the problem with a larger number of bits.  
`big_hints_out_file` is an output file, where the CNF hints to add to the top of `big_cnf` will be written by the program

In [29]:
small_cnf = 'EMD/8bitcnf.cnf'
small_sat_output = 'EMD/8bitout.out'
big_cnf = 'EMD/64bitcnf.cnf'
big_hints_out_file = 'EMD/64bit-hints.txt'

## Read in the 8-bit CNF File
Read in the 8-bit CNF, putting comments (lines where the first char is 'c') into an array.
These comments contain CBMC's annotations on which CNF variables correspond to which variables in the initial C program. We want to figure out a mapping between CNF variables and C variables which are AIs

In [30]:
with open(small_cnf) as f:
    data = f.read()
lines_data = data.split('\n')
c_lines = []
for line in lines_data:
    try:
        if line[0] == 'c':
            c_lines.append(line)
    except IndexError:
        continue

Each of these comments corresponds to one variable + program state combination. We want only those ones which are AIs; i.e the variable name contains "AI_"

This is guaranteed based on the convention that we name only AI variables with this prefix in our C program. If that convention is not followed, this script will not function properly.

In [31]:
# Each comment line represents a conversion from a variable in the program into cnf variables; we care about the ones with an AI in
ai_lines = []
for line in c_lines:
    if "AI_" in line:
        ai_lines.append(line)
len(ai_lines)

75

Now, we make a mapping from variable names to 8-bit cnf variables. 

Note that the CNF variables will change depending on the number of bits in the program, bit the C variables won't.

In [32]:
cnf_map_8bit = {}
for line in ai_lines:
    # Variable name is the second element in a split on whitespace
    split_line = line.split(" ")
    v_name = split_line[1]
    cnf_vars = []
    i = 2
    while i<len(split_line):
        cnf_vars.append(split_line[i])
        i+=1
    cnf_map_8bit[v_name] = cnf_vars

Now we know which program variables correspond to which cnf variables in the 8 bit assignment, let's find out whether they are set to true or false in the SAT output.

NB: This code is assuming that you're using output from the Kissat solver; if you're using a different SAT solver you'll need to modify the file being passed in slightly.

An example of a few lines of correctly formatted counter example look like this:
```
s SATISFIABLE
v 1 -2 -3 4 -5 -6 7 8 9 -10 -11 -12 -13 -14 -15 -16 -17 -18 -19 -20 -21 -22
v -23 -24 -25 -26 -27 -28 -29 -30 -31 -32 -33 34 -35 -36 37 -38 -39 40 -41 -42
v 43 -44 -45 46 -47 -48 49 -50 -51 52 -53 -54 55 56 57 -58 -59 -60 -61 -62 63
...
```
Note the first line with the solver result, and the `v ` in front of each line of variable assignments.

If you're piping directly from kissat in the terminal, also make sure to delete the comment lines before and after the SAT output (as these will cause errors)

In [33]:
with open(small_sat_output) as f:
    counter_data = f.read()
assignments = counter_data.split('\n')
len(assignments)

856

Now we create a boolean mapping against CNF variables; i.e which CNF variables are set to true in the SAT counter example provided, and which are set to False

In [34]:
var_assignments = {}
for assignment_line in assignments[1:]:
    for assignment in assignment_line.split(' ')[1:]:
        if assignment[0] == '-':
            var_assignments[assignment[1:]] = False
        else:
            var_assignments[assignment] = True

Combining the mapping of C variables to CNF variables, and CNF variables to truth values, we can figure out the mask for each of the C program variables in the larger bit version of the problem. That is, whether the CNF variable in each position in the CBMC output should be set to True or False

In [35]:
# Use these two mappings to create a mapping between program variables and truth values
truth_assignments_8bit = {}
for key in cnf_map_8bit.keys():
    val = cnf_map_8bit[key]
    truth_list = []
    for var in val:
        truth_list.append(var_assignments[var])
    truth_assignments_8bit[key] = truth_list
    assert(len(truth_list) == len(val))

Now we know what the assignments on the small version were for our satisfying assignment, figure out which CNF variables they correspond to in the larger cnf.

The next three cells are pre-processing steps, similar to those carried out for the smaller CNF

In [36]:
with open(big_cnf) as f:
    big_data = f.read()
lined_big_data = big_data.split('\n')
lined_big_data

['p cnf 255552 159231',
 '1 -3 0',
 '2 -3 0',
 '-1 -2 3 0',
 '1 -5 0',
 '2 -5 0',
 '4 -5 0',
 '-1 -2 -4 5 0',
 '1 -6 0',
 '2 -6 0',
 '-4 -6 0',
 '-1 -2 4 6 0',
 '1 -8 0',
 '2 -8 0',
 '4 -8 0',
 '7 -8 0',
 '-1 -2 -4 -7 8 0',
 '1 -10 0',
 '2 -10 0',
 '4 -10 0',
 '7 -10 0',
 '9 -10 0',
 '-1 -2 -4 -7 -9 10 0',
 '1 -11 0',
 '2 -11 0',
 '4 -11 0',
 '7 -11 0',
 '-9 -11 0',
 '-1 -2 -4 -7 9 11 0',
 '1 -13 0',
 '2 -13 0',
 '4 -13 0',
 '12 -13 0',
 '-1 -2 -4 -12 13 0',
 '1 -14 0',
 '2 -14 0',
 '4 -14 0',
 '-12 -14 0',
 '-1 -2 -4 12 14 0',
 '1 -16 0',
 '2 -16 0',
 '4 -16 0',
 '-12 -16 0',
 '15 -16 0',
 '-1 -2 -4 12 -15 16 0',
 '1 -18 0',
 '2 -18 0',
 '4 -18 0',
 '-12 -18 0',
 '15 -18 0',
 '17 -18 0',
 '-1 -2 -4 12 -15 -17 18 0',
 '1 -19 0',
 '2 -19 0',
 '4 -19 0',
 '-12 -19 0',
 '15 -19 0',
 '-17 -19 0',
 '-1 -2 -4 12 -15 17 19 0',
 '-12 -20 0',
 '1 -20 0',
 '2 -20 0',
 '4 -20 0',
 '15 -20 0',
 '-1 -2 -4 12 -15 20 0',
 '1 -21 0',
 '2 -21 0',
 '4 -21 0',
 '-12 -21 0',
 '-15 -21 0',
 '-1 -2 -4 12 15

In [37]:
big_c_lines = []
for line in lined_big_data:
    try:
        if line[0] == 'c':
            big_c_lines.append(line)
    except IndexError:
        continue

In [38]:
big_ai_lines = []
for line in big_c_lines:
    if "AI_" in line:
        big_ai_lines.append(line)
len(big_ai_lines)

75

Now that we have the CNF variables corresponding to the C variables in the larger CNF, we figure out which CNF variables they correspond to in the smaller CNF.

In [39]:
cnf_map_32bit = {}
# Make a dictionary (mapping) from variable names (which will be the same) to 8-bit cnf vars (which will be different)
for line in big_ai_lines:
    # Variable name is the second element in a split on whitespace
    split_line = line.split(" ")
    v_name = split_line[1]
    cnf_vars = []
    i = 2
    while i<len(split_line):
        cnf_vars.append(split_line[i])
        i+=1
    cnf_map_32bit[v_name] = cnf_vars

For each of these CNF variables, determine whether they should be true or false, and write to a file the appropriate clause (with a 0 on the end)

In [40]:
with open(big_hints_out_file, 'w') as f:
    for key in cnf_map_32bit.keys():
        val = cnf_map_32bit[key]
        truth_list = truth_assignments_8bit[key]
        for (idx,truth) in enumerate(truth_list):
            if truth:
                f.write(val[idx] + " 0\n")
            else:
                f.write("-" + val[idx] + " 0\n")

## Using the hints
To use the hints, you can paste them verbatim into the top of the larger CNF file (just below the problem statement). Make sure to also adjust the number of clauses in the problem statement if using a SAT solver where these numbers are used, as you will have just added a number of clauses to the CNF!